In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from sklearn import metrics
from tensorflow import keras
from keras.layers import Dense, Dropout, TextVectorization, Flatten
from keras import  Model
import keras
import tensorflow as tf
import evaluate

/opt/anaconda3/envs/discoursetbi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Processing

In [2]:
# Create dataframe from csv
baseline_data = pd.read_csv('baseline.csv')

In [3]:
baseline_data = baseline_data.drop("Unnamed: 0", axis=1)
baseline_data['labels'] = baseline_data['labels'].astype('category')
baseline_data['labels'] = baseline_data['labels'].cat.codes

In [5]:
X = baseline_data['text']
Y = baseline_data['labels'].values    

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, random_state= 42)

In [6]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [7]:
train_samples = [x for x in X_train]
train_labels = [y for y in y_train]

test_samples = [x for x in X_test]
test_labels = [y for y in y_test]

In [8]:
# Vectorize texts
vectorizer = TextVectorization(max_tokens=1000, output_mode= 'multi_hot', pad_to_max_tokens=True)
vectorizer.adapt(train_samples)
voc = vectorizer.get_vocabulary()

In [9]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_test  = vectorizer(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_test = np.array(test_labels)

### Basic RNN Model

In [22]:
input = keras.Input((1000,))
x = Flatten()(input)
x = Dense(16, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(2, activation='sigmoid')(x)
model = Model(inputs = input, outputs= output)
model.summary()

# Train the model
callback = keras.callbacks.EarlyStopping()
opt = keras.optimizers.AdamW(learning_rate=0.05)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["binary_accuracy", "f1_score"])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test),callbacks=[callback])

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │        16,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,050 (62.70 KB)

 Trainable params: 16,050 (62.70 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - binary_accuracy: 0.4634 - f1_score: 0.4387 - loss: 0.8672 - val_binary_accuracy: 0.4722 - val_f1_score: 0.2800 - val_loss: 0.7331
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - binary_accuracy: 0.6892 - f1_score: 0.7706 - loss: 0.5468 - val_binary_accuracy: 0.5556 - val_f1_score: 0.7214 - val_loss: 0.7034
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - binary_accuracy: 0.8512 - f1_score: 0.9762 - loss: 0.3462 - val_binary_accuracy: 0.4722 - val_f1_score: 0.4985 - val_loss: 0.6740
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - binary_accuracy: 0.9517 - f1_score: 0.9627 - loss: 0.1955 - val_binary_accuracy: 0.5278 - val_f1_score: 0.5500 - val_loss: 0.6694
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - binary_accuracy: 0.9465 - f1_score: 0.9520 - loss: 0.1208 - val_binary_accuracy: 0.5833 - val_f1_score: 0.5325 - val_loss: 0.6523
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - binary_accuracy: 0.9530 - f1_score: 0.9526 -

## BERT Model

In [11]:
dataset = Dataset.from_pandas(baseline_data)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.25)

Map: 100%|██████████| 69/69 [00:00<00:00, 4479.64 examples/s]


In [14]:
metric = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    eval_strategy="epoch",
)

In [17]:
bert_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [18]:
bert_trainer.train()

 33%|███▎      | 7/21 [00:11<00:19,  1.37s/it]

{'eval_loss': 0.697288990020752, 'eval_accuracy': 0.5555555555555556, 'eval_runtime': 1.4636, 'eval_samples_per_second': 12.299, 'eval_steps_per_second': 2.05, 'epoch': 1.0}


                                               
 67%|██████▋   | 14/21 [00:23<00:10,  1.44s/it]

{'eval_loss': 0.7073937654495239, 'eval_accuracy': 0.5555555555555556, 'eval_runtime': 1.4263, 'eval_samples_per_second': 12.62, 'eval_steps_per_second': 2.103, 'epoch': 2.0}


                                               
100%|██████████| 21/21 [00:38<00:00,  1.84s/it]

{'eval_loss': 0.7170486450195312, 'eval_accuracy': 0.5555555555555556, 'eval_runtime': 1.0806, 'eval_samples_per_second': 16.658, 'eval_steps_per_second': 2.776, 'epoch': 3.0}
{'train_runtime': 38.6185, 'train_samples_per_second': 3.962, 'train_steps_per_second': 0.544, 'train_loss': 0.6765328816005162, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=0.6765328816005162, metrics={'train_runtime': 38.6185, 'train_samples_per_second': 3.962, 'train_steps_per_second': 0.544, 'total_flos': 40255991470080.0, 'train_loss': 0.6765328816005162, 'epoch': 3.0})

In [19]:
bert_trainer.evaluate(tokenized_datasets['train'])

100%|██████████| 7/7 [00:02<00:00,  2.38it/s]


{'eval_loss': 0.5243889093399048,
 'eval_accuracy': 0.7647058823529411,
 'eval_runtime': 3.476,
 'eval_samples_per_second': 14.672,
 'eval_steps_per_second': 2.014,
 'epoch': 3.0}

In [20]:
accuracy = evaluate.load("accuracy")